# Train

In [11]:
import numpy as np
import pandas as pd
import cv2
%matplotlib inline
import matplotlib.pyplot as plt
import pickle
import csv
import tensorflow as tf
from PIL import Image
import os
import keras
import sys

# Import architectures
from keras.applications import densenet
from keras.applications import xception
from keras.applications import resnet_v2

from keras.preprocessing import image
from tensorflow.keras.preprocessing import image as im
import time
import datetime

from keras.preprocessing.image import ImageDataGenerator
from keras.models import load_model
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, Callback
from keras import regularizers
from keras import backend as K
from sklearn.preprocessing import OneHotEncoder

from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.models import Sequential, Model
from keras import layers
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Input, AveragePooling2D
from keras.layers import Activation, Flatten, Dense, Dropout

In [12]:
# Train phase

tf.keras.backend.set_learning_phase(1) 

In [13]:
# Choose GPU

os.environ["CUDA_VISIBLE_DEVICES"] = "2"

In [14]:
# 1) Path to train and test datasets
train_dir = "/home/linnik/Downloads/Face/vggface2_train/"   
test_dir = "/home/linnik/Downloads/Face/vggface2_test/"


# 2) Target width and height of input images, number of classes, number of train and test images, batch size, epochs
img_height, img_width = 250, 250
n_classes = len(os.listdir(train_dir))
nb_train_samples = sum([len(os.listdir(train_dir + folder)) for folder in os.listdir(train_dir)])
nb_test_samples = sum([len(os.listdir(test_dir + folder)) for folder in os.listdir(test_dir)])
batch_size = 128
epochs = 100


# 3) Compile parametrs
loss_ = 'categorical_crossentropy'
optimizer_ = 'adam'
metrics_ = ['acc']

train_datagen = ImageDataGenerator(
    rescale = 1.0 / 255,
    zoom_range = 0.2,
    rotation_range = 5,
    width_shift_range = img_width // 5,
    height_shift_range = img_height // 5,
    horizontal_flip = True,
    vertical_flip = False)

test_datagen = ImageDataGenerator(rescale = 1.0 / 255)

In [ ]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size = (img_height, img_width),
    batch_size = batch_size,
    class_mode = 'categorical',
    classes = os.listdir(test_dir))


test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size = (img_height, img_width),
    batch_size = batch_size,
    class_mode = 'categorical',
    classes = os.listdir(test_dir))

In [ ]:
dic_test = test_generator.class_indices

In [ ]:
def build_model_Xception():
    base_model = xception.Xception(include_top = True, 
                                      weights = None,
                                      input_tensor = None,
                                      input_shape = (img_height, img_width, 3),
                                      pooling = None,
                                      classes = n_classes)
  
    for layer in base_model.layers:
        layer.trainable = True
      
    model = Model(inputs = base_model.input, outputs = base_model.output)
    return model


model = build_model_Xception()
model.compile(loss = loss_, optimizer = optimizer_, metrics = metrics_)

In [21]:
# Callbacks

early_stop = EarlyStopping(monitor = 'val_loss', patience = 8, verbose = 1, min_delta = 1e-4)
reduce_lr = ReduceLROnPlateau(monitor = 'val_loss', factor = 0.1, patience = 4, verbose = 1, min_delta = 1e-4)

callbacks_list = [early_stop, reduce_lr]

In [ ]:
%%time

# Train model

model_history = model.fit_generator(
    train_generator,
    epochs = epochs,
    validation_data = test_generator,
    validation_steps = nb_test_samples // batch_size,
    callbacks = callbacks_list,
    steps_per_epoch = nb_train_samples // batch_size)

# Test

In [50]:
# Test phase

tf.keras.backend.set_learning_phase(0)

In [51]:
def Accuracy(test_dir, model, dic):
    n_classes = len(os.listdir(test_dir))
    dic_rev = {v:k for k, v in dic.items()}
    
    right = 0
    wrong = 0

    for folder in os.listdir(test_dir):
        for img in os.listdir(test_dir + "/" + folder):
            imag = im.load_img(test_dir + "/" + folder + "/" + img, target_size = (img_height, img_width))

            photo_ar = im.img_to_array(imag)
            photo_ar = np.expand_dims(photo_ar, axis = 0)
            photo_ar /= 255

            ans = np.argmax(model.predict(photo_ar))


            if folder == dic_rev[ans]:
                right += 1
            else:
                wrong += 1
    
    
    accuracy = right/(right + wrong)
    return accuracy

In [ ]:
%%time

accuracy = Accuracy(test_dir, model, dic_test)
print("Accuracy on test set is", accuracy)